# 制作评估语料

注意输入的 Context tokens 长度要小于最大模型最大生成序列长度的一半！

## 代码准备

### Imporings

In [1]:
import json
import math
import os
import sys
from contextlib import ExitStack
from datetime import timedelta
from functools import partial
from glob import glob, iglob
from itertools import chain, cycle, islice, count
from multiprocessing import Pool
from time import time

import numpy as np
import sentencepiece as spm
from tqdm.auto import tqdm


### Constants

In [2]:
SEQ_LENGTH = 1024
MIN_CTX_LEN = 128

SPM_MODEL = '../data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'


SP = spm.SentencePieceProcessor()
SP.load(SPM_MODEL)

True

### Functions

In [3]:
def text_files_line_iterator(paths):
    return chain.from_iterable(
        open(path)
        for path
        in tqdm(paths, '[iter files]', unit='file')
    )


def single_text_file_line_count(path, show_progress_bar=False):
    with open(path) as fd:
        iterable = tqdm(fd) if show_progress_bar else fd
        return sum(1 for _ in iterable)
        

def text_files_line_count(paths):
    try:
        total = len(paths)
    except (AttributeError, TypeError):
        total = None
    with Pool() as pool:
        it = pool.imap_unordered(
            single_text_file_line_count,
            tqdm(paths, '[map files ]', unit='file')
        )
        return sum(c for c in tqdm(it, '[sum files ]', unit='file', total=total))


def proc_line(line):
    result = []
    line = line.strip()
    if not line:
        return result
    paragraphs = json.loads(line)
    text = ''
    n_text = 0
    for sentence in chain.from_iterable(paragraphs):
        sentence = sentence.strip()
        if not sentence:
            continue
        n_sentence = len(SP.encode_as_ids(sentence))
        if n_text + n_sentence < SEQ_LENGTH + MIN_CTX_LEN // 2:
            text += sentence
            n_text += n_sentence
        else:
            result.append({'text': text, 'length': n_text})
            text = sentence
            n_text = n_sentence
    if n_text:
        result.append({'text': text, 'length': n_text})
    return result
    

## 语料文件

### 输入文件

### 列出输入文件

In [4]:
INPUT_FILES = [
    path
    for path in tqdm(iglob(
        "/nfs/server01_public/data/gpt2/output/gpt2_huamei_corpus.json.8g/**/*.*",
        recursive=True
    ))
    if os.path.isfile(path) and os.path.splitext(path)[1].lower() in ('.jsonl', '.jsonlines', 'json', 'jsonline')
]
print(f'源语料文件数：{len(INPUT_FILES):,d}')


源语料文件数：379


### 统计输入文件总行数

In [5]:
%%time

total_lines = text_files_line_count(INPUT_FILES)
print(f'源语料行数：{total_lines:,d}')



源语料行数：1,710,236
CPU times: user 142 ms, sys: 48 ms, total: 190 ms
Wall time: 2.5 s


### 输出文件

In [6]:
OUTPUT_FILE = '../data/gpt2_huamei_corpus_emotion.jsonl'